# Import libraries

In [ ]:
import os
import random
import numpy as np
import pandas as pd
from math import sqrt
import math
import os
import random

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim

In [ ]:
!pip install autoimpute
from autoimpute.imputations import MultipleImputer

# Missingness method

In [ ]:
def missing_method(test_data, train_data, num_embeddings) :
    
    # test data
    test_data = test_data.copy()
    test_rows, test_cols = test_data.shape
    test_cols -= num_embeddings

    # train data
    train_data = train_data.copy()
    train_rows, train_cols = train_data.shape
    train_cols -= num_embeddings

    # missingness threshold
    t = 0.2

    # uniform random vector, missing values where v<=t
    # test data corruption
    # embedding columns do not have any missing values
    v = np.random.uniform(size=(test_rows, test_cols))
    embeddings_mask = np.zeros((test_rows, num_embeddings), dtype=bool)
    mask = (v<=t)
    mask = np.c_[mask, embeddings_mask]
    test_data[mask] = np.NAN

    # train data corruption - this is used for training MultipleImputer for imputing mean/median values in the dataset
    v_train = np.random.uniform(size=(train_rows, train_cols))
    embeddings_mask = np.zeros((train_rows, num_embeddings), dtype=bool)
    mask_train = (v_train <= t)
    mask_train = np.c_[mask_train, embeddings_mask]
    train_data[mask_train] = np.NAN
        
    return test_data, train_data, mask

# Imputation Methods

## Autoencoder

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, dim):
        super(Autoencoder, self).__init__()
        self.dim = dim
        
        self.drop_out = nn.Dropout(p=0.2)
        
        # encoder architecture
        self.encoder = nn.Sequential(
            nn.Linear(dim, int(dim*0.7)),
            nn.Tanh(),
            nn.Linear(int(dim*0.7), int(dim*0.5)),
            nn.Tanh(),
            nn.Linear(int(dim*0.5), int(dim*0.2))
        )
            
        # decoder architecture
        self.decoder = nn.Sequential(
            nn.Linear(int(dim*0.2), int(dim*0.5)),
            nn.Tanh(),
            nn.Linear(int(dim*0.5), int(dim*0.7)),
            nn.Tanh(),
            nn.Linear(int(dim*0.7), dim)
        )
        
    def forward(self, x):
        x = x.view(-1, self.dim)

        # adding dropout to introduce input corruption during training
        x_missed = self.drop_out(x)
        
        z = self.encoder(x_missed)
        out = self.decoder(z)
        
        out = out.view(-1, self.dim)
        
        return out

In [ ]:
def training(num_epochs, model, train_loader, criterion, optimizer):
  for epoch in range(num_epochs):
      loss = 0
      for i, batch_features in enumerate(train_loader):
          # load it to the active device
          batch_features = batch_features.to(device)
        
          # reset the gradients back to zero
          optimizer.zero_grad()
        
          # compute reconstructions
          outputs = model(batch_features)
        
          # compute training reconstruction loss
          train_loss = criterion(outputs, batch_features)
        
          # compute accumulated gradients
          train_loss.backward()
        
          # perform parameter update based on current gradients
          optimizer.step()
        
          # add the mini-batch training loss to epoch loss
          loss += train_loss.item()
    
      # compute the epoch training loss
      loss = loss / len(train_loader)
    
      #  display the epoch training loss
      print("epoch : {}/{}, recon loss = {:.8f}".format(epoch + 1, num_epochs, loss))

## Mean and Median Imputation

In [ ]:
# takes the test dataset and the dataset containing the average/median values by category
# imputes the values found in the dataset by catogory in place of the NaN values in the test dataset
# returns pandas DataFrame

def impute_traditional_by_category(X_test, df, indices):
  for i in indices:
    temp1 = X_test.loc[[i]]
    temp2 = df[df['fdc_id'] == i]
    del temp2['fdc_id']
    for col in temp1.columns:
      if math.isnan(temp1[col].values[0]):
        temp1[col] = temp2[col].values[0]
    X_test.loc[[i]] = temp1

  return X_test

In [ ]:
# uses a multiple imputer to fill the NaN values in the test dataset, calculates mean/median by column
# returns pandas DataFrame

def impute_traditional_all_foods(train, test, method):
  imputer = MultipleImputer(1, strategy=method, return_list=True)
  imputer.fit(train)
  data = imputer.transform(test)
  return data[0][1]

# Evaluation

In [ ]:
def rmse_error(test_data, imputed_data, num_cols, mask):
  rmse_sum = 0

  for i in range(num_cols):
    y_actual = test_data[:,i][mask[:,i]]
    y_predicted = imputed_data[:,i][mask[:,i]]

    rmse = sqrt(mean_squared_error(y_actual, y_predicted))
    rmse_sum += rmse

  return rmse_sum

# Data Preparation

In [ ]:
num_epochs = 200
test_size = 0.2
use_cuda = False
batch_size  = 1

data_mean = 'datasets/average datasets/traditionalMethods/food_mean.csv'
data_median = 'datasets/average datasets/traditionalMethods/food_median.csv'

In [ ]:
df_mean = pd.read_csv(data_mean)
del df_mean['food_category_id']
del df_mean['Unnamed: 0']
df_mean.describe()

,fdc_id,1002,1003,1004,1005,1007,1008,1009,1010,1011,1012,1013,1014,1024,1032,1039,1050,1051,1062,1063,1075,1079,1082,1084,1085,1087,1089,1090,1091,1092,1093,1094,1095,1097,1098,1100,1101,1102,1103,1105,...,1323,1325,1329,1330,1331,1333,1334,1335,1404,1405,1406,1409,1411,1414,2003,2004,2005,2006,2007,2008,2009,2010,2012,2013,2014,2015,2016,2018,2019,2020,2021,2022,2023,2024,2025,2026,2028,2029,2032,2033
count,1.310500e+04,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,...,13105.000000,13105.0,13105.000000,13105.000000,1.310500e+04,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.0,13105.0,13105.0,13105.0,13105.000000,13105.0,13105.000000,13105.000000,13105.000000,13105.000000,13105.0,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000
mean,4.807517e+05,0.243665,1.942243,1.824697,0.164999,0.472742,1.999714,2.561062,0.147034,0.118187,0.120046,0.046356,0.008754,0.000476,0.681114,0.728577,0.049253,8.349769,8.369340,0.055411,0.002459,0.350355,0.005555,0.021892,0.106772,27.659061,0.476038,14.595597,55.746662,125.003892,133.394887,14.730179,0.384163,2.983579,0.077504,12.187135,0.149979,42.944411,0.840258,2.086684,...,0.002480,0.0,0.001246,0.000236,7.630675e-07,0.000002,0.000065,0.000017,0.016719,0.000279,0.001158,0.000021,0.000652,0.0,0.0,0.0,0.0,0.000093,0.0,0.000153,0.002787,0.000194,0.005167,0.0,0.000170,0.000018,0.020393,0.001910,0.000021,0.000090,0.000004,0.000217,0.000025,0.000044,0.000039,0.000110,0.298283,0.004426,0.005723,0.022570
std,2.544018e+05,0.220034,5.131636,1.626531,0.445364,1.407237,2.322706,7.750389,1.676538,0.170909,0.170768,0.066607,0.022466,0.001082,1.547442,1.655274,0.088366,6.015826,9.725673,0.336683,0.003077,0.866652,0.009656,0.039604,0.268325,36.719285,1.102398,36.788753,104.198028,312.532122,574.034787,46.850594,0.685919,9.489527,0.216968,77.766980,0.408316,135.856952,0.727050,2.920551,...,0.004219,0.0,0.002653,0.000311,2.094377e-06,0.000006,0.000102,0.000024,0.062646,0.000978,0.001561,0.000030,0.001585,0.0,0.0,0.0,0.0,0.000410,0.0,0.000214,0.003980,0.000502,0.017287,0.0,0.000669,0.000097,0.073219,0.008556,0.000029,0.000180,0.000018,0.000493,0.000062,0.000093,0.000208,0.000393,0.359319,0.006378,0.008247,0.037044
min,3.198770e+05,0.000000,0.000000,0.111598,0.000000,0.000945,0.259574,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000787,1.080851,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019685,0.001024,0.000000,0.000000,0.000000,0.000000,0.000000,0.000315,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.256480e+05,0.039103,0.032765,0.213767,0.029723,0.126017,0.852374,0.000000,0.014362,0.002979,0.001510,0.000000,0.000000,0.000000,0.000000,0.000000,0.002864,7.083392,3.566526,0.009194,0.000000,0.000399,0.000000,0.000000,0.011009,2.582981,0.089350,1.771225,5.056801,19.483948,11.090738,0.000000,0.034903,0.000000,0.002225,0.000000,0.001896,0.000000,0.101719,0.000000,...,0.000156,0.0,0.000007,0.000024,0.000000e+00,0.000000,0.000000,0.000000,0.00421

In [ ]:
df_median = pd.read_csv(data_median, index_col=0)
del df_median['food_category_id']
df_median.describe()

,fdc_id,1002,1003,1004,1005,1007,1008,1009,1010,1011,1012,1013,1014,1024,1032,1039,1050,1051,1062,1063,1075,1079,1082,1084,1085,1087,1089,1090,1091,1092,1093,1094,1095,1097,1098,1100,1101,1102,1103,1105,...,1323,1325,1329,1330,1331,1333,1334,1335,1404,1405,1406,1409,1411,1414,2003,2004,2005,2006,2007,2008,2009,2010,2012,2013,2014,2015,2016,2018,2019,2020,2021,2022,2023,2024,2025,2026,2028,2029,2032,2033
count,1.310500e+04,13105.0,13105.000000,13105.000000,13105.0,13105.000000,13105.0,13105.000000,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.000000,13105.0,13105.0,13105.0,13105.000000,13105.0,13105.0,13105.0,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.0,13105.000000,13105.000000,13105.000000,13105.000000,13105.0,13105.000000,13105.000000,13105.0,13105.0,...,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0,13105.0
mean,4.807517e+05,0.0,2.417543,0.142532,0.0,0.013915,0.0,3.310736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.865044,0.0,0.0,0.0,0.368859,0.0,0.0,0.0,13.674781,0.424638,14.934300,43.813277,133.149180,0.0,19.522549,0.288790,3.508661,0.085287,0.0,0.157440,34.636780,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,2.544018e+05,0.0,7.085115,0.438829,0.0,0.115348,0.0,10.530080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.170618,0.0,0.0,0.0,1.173188,0.0,0.0,0.0,43.493809,1.350597,47.499817,139.351873,423.492346,0.0,62.093135,0.918521,11.159596,0.271264,0.0,0.500751,110.165239,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,3.198770e+05,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,3.256480e+05,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,3.314100e+05,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,7.474680e+05,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,1.105897e+06,0.0,23.300000,2.410000,0.0,0.970000,0.0,36.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.300000,0.0,0.0,0.0,4.100000,0.0,0.0,0.0,152.000000,4.720000,166.000000,487.000000,1480.000000,0.0,217.000000,3.210000,39.000000,0.948000,0.0,1.750000,385.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0

In [ ]:
device = torch.device("cuda" if use_cuda else "cpu")

# 20 Columns

In [ ]:
data_path = 'datasets/average datasets/byColumnsWithEmbeddings/20_columns_embeddings.csv'
num_embeddings = 5
num_columns = 20

In [ ]:
dataset = pd.read_csv(data_path, index_col=0)
dataset[dataset.notna().all(axis=1)].describe()

,fdc_id,1004,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1002,1093,1079,1003,1009,1102,1265,1266,1264,0,1,2,3,4
count,1.310500e+04,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000
mean,4.807517e+05,1.824697,27.659061,14.595597,125.003892,0.384163,55.746662,0.476038,0.077504,0.149979,8.349769,0.472742,0.243665,133.394887,0.350355,1.942243,2.561062,42.944411,0.308332,0.116491,0.065003,21488.189968,176.402583,0.119704,-0.093912,-0.048493
std,2.544018e+05,7.838466,99.924214,45.754175,382.322293,1.055340,153.432660,1.483360,0.264892,0.515558,23.549114,4.416751,0.999601,1736.846193,1.186155,6.583455,9.446549,187.200801,1.605577,0.597369,0.559044,254447.573253,3958.010969,1.112342,0.694392,0.598292
min,3.198770e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-139555.939272,-7413.329674,-3.349638,-1.585647,-1.527263
25%,3.256480e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-133692.806088,-2328.507401,-0.796369,-0.667107,-0.477074
50%,3.314100e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-127823.125712,-153.274226,-0.002647,-0.011945,-0.071942
75%,7.474680e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,288262.097477,3032.870912,0.983044,0.441140,0.323268
max,1.105897e+06,99.400000,1390.000000,386.000000,2520.000000,7.660000,997.000000,12.700000,1.920000,7.870000,96.700000,99.600000,13.200000,40700.000000,11.500000,79.900000,43.900000,1790.000000,22.800000,8.610000,17.000000,646711.616853,7635.496843,2.647689,1.869347,1.616847


In [ ]:
data = dataset
train_data, test_data = train_test_split(data, test_size=test_size)

In [ ]:
train_data.describe()

,fdc_id,1004,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1002,1093,1079,1003,1009,1102,1265,1266,1264,0,1,2,3,4
count,1.048400e+04,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000
mean,4.822540e+05,1.755201,27.743609,14.484309,124.078024,0.380677,55.346433,0.474776,0.077031,0.148934,8.414429,0.476104,0.234480,139.155284,0.355341,1.915318,2.559018,42.233441,0.308766,0.118179,0.063822,22990.229720,203.642129,0.120423,-0.094587,-0.045482
std,2.559578e+05,7.687655,101.585144,45.636608,381.855342,1.053683,153.490134,1.485049,0.263929,0.514145,23.687789,4.515724,0.967373,1822.974657,1.197866,6.542063,9.447906,184.659671,1.595443,0.603438,0.523031,256004.025403,3967.478183,1.109581,0.694324,0.598217
min,3.198770e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-139555.939272,-7413.329674,-3.349638,-1.585647,-1.481988
25%,3.256225e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-133718.782631,-2326.205329,-0.788413,-0.671057,-0.476737
50%,3.314150e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-127818.032271,-111.096255,-0.002088,-0.011191,-0.071399
75%,7.475152e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,288310.230486,3047.830480,0.982963,0.441077,0.330094
max,1.105897e+06,99.400000,1390.000000,386.000000,2510.000000,7.660000,997.000000,10.800000,1.920000,7.870000,96.700000,99.600000,13.000000,40700.000000,11.400000,79.900000,43.900000,1790.000000,22.800000,8.610000,16.800000,646711.616853,7635.496843,2.647689,1.869347,1.597387


In [ ]:
test_data.describe()

,fdc_id,1004,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1002,1093,1079,1003,1009,1102,1265,1266,1264,0,1,2,3,4
count,2.621000e+03,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000
mean,4.747426e+05,2.102678,27.320870,15.040748,128.707364,0.398108,57.347577,0.481087,0.079392,0.154159,8.091129,0.459294,0.280404,110.353300,0.330408,2.049947,2.569241,45.788287,0.306598,0.109738,0.069724,15480.030960,67.444400,0.116832,-0.091213,-0.060537
std,2.480369e+05,8.410532,93.001944,46.227551,384.235236,1.062026,153.221347,1.476856,0.268750,0.521250,22.988673,3.997199,1.118717,1338.044656,1.138109,6.746680,9.442917,197.048894,1.645802,0.572513,0.684504,248081.146906,3918.771433,1.123528,0.694789,0.598559
min,3.198820e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-139550.845832,-7411.367800,-2.816578,-1.537302,-1.527263
25%,3.257310e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-133608.254984,-2345.183653,-0.820487,-0.646966,-0.482865
50%,3.314030e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-127830.256528,-330.340365,-0.006454,-0.014591,-0.076192
75%,7.472940e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,288084.845758,2974.013474,0.984178,0.441246,0.321993
max,1.105887e+06,82.600000,938.000000,369.000000,2520.000000,7.050000,986.000000,12.700000,1.840000,4.010000,96.000000,99.400000,13.200000,40100.000000,11.500000,48.100000,42.500000,1610.000000,22.200000,7.420000,17.000000,646701.429977,7630.592063,2.647168,1.869343,1.616847


In [ ]:
indices = test_data['fdc_id']
indices

767      321893
11351    790103
7753     333420
3071     325328
1309     322529
          ...  
3525     326013
8207     334668
11660    790480
5439     329186
2603     324520
Name: fdc_id, Length: 2621, dtype: int64

In [ ]:
del train_data['fdc_id']
del test_data['fdc_id']
columns = train_data.columns

In [ ]:
train_data = train_data.to_numpy()
test_data.fillna(0, inplace=True)
test_data = test_data.to_numpy()
data = dataset.values
rows, cols = data.shape
cols -= 1

y_test = test_data.copy()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
missed_data, missed_data_train, mask = missing_method(test_data, train_data, num_embeddings)
pd.DataFrame(missed_data).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
count,2128.000000,2103.000000,2081.000000,2088.000000,2107.000000,2127.000000,2096.000000,2105.000000,2098.000000,2116.000000,2100.000000,2067.000000,2093.000000,2115.000000,2080.000000,2085.000000,2138.000000,2111.000000,2069.000000,2088.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000
mean,2.127702,28.920114,14.914512,126.250000,0.408519,58.964269,0.489957,0.079634,0.159527,8.198341,0.446043,0.306473,86.846632,0.330875,2.031240,2.564427,45.905753,0.324708,0.111738,0.067591,15480.030960,67.444400,0.116832,-0.091213,-0.060537
std,8.600823,95.757893,45.769414,380.203401,1.074592,155.364526,1.495226,0.270365,0.526523,23.213875,3.877333,1.184570,879.867413,1.127328,6.722095,9.429371,198.052557,1.712762,0.574624,0.700947,248081.146906,3918.771433,1.123528,0.694789,0.598559
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-139550.845832,-7411.367800,-2.816578,-1.537302,-1.527263
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-133608.254984,-2345.183653,-0.820487,-0.646966,-0.482865
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-127830.256528,-330.340365,-0.006454,-0.014591,-0.076192
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,288084.845758,2974.013474,0.984178,0.441246,0.321993
max,82.600000,938.000000,369.000000,2030.000000,6.630000,986.000000,12.700000,1.840000,4.010000,96.000000,99.400000,13.200000,38200.000000,11.000000,48.100000,42.500000,1610.000000,22.200000,7.420000,17.000000,646701.429977,7630.592063,2.647168,1.869343,1.616847


In [ ]:
# traditional methods test sets - impute across all foods
X_train = pd.DataFrame(missed_data_train, columns = columns)
X_test_mean_all = pd.DataFrame(missed_data, columns = columns)
X_test_median_all = pd.DataFrame(missed_data, columns = columns)

X_test_mean_all = impute_traditional_all_foods(X_train, X_test_mean_all, 'mean')
X_test_median_all = impute_traditional_all_foods(X_train, X_test_median_all, 'median')

In [ ]:
# Traditional methods test sets - impute by food category
X_test_mean = pd.DataFrame(missed_data, columns = columns, index=indices)
X_test_median = X_test_mean.copy()
df_mean_cat = df_mean[df_mean['fdc_id'].isin(indices)]
X_test_mean = impute_traditional_by_category(X_test_mean, df_mean_cat, indices)
df_median_cat = df_median[df_median['fdc_id'].isin(indices)]
X_test_median = impute_traditional_by_category(X_test_median, df_median_cat, indices)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)
X_test_mean = scaler.transform(X_test_mean)
X_test_median = scaler.transform(X_test_median)
X_test_mean_all = scaler.transform(X_test_mean_all)
X_test_median_all = scaler.transform(X_test_median_all)

y_test = scaler.transform(y_test)

In [ ]:
# datasets without embeddings for autoencoder
train_data_noEmbeddings = pd.DataFrame(train_data, columns = columns)
train_data_noEmbeddings = np.array(train_data_noEmbeddings.iloc[:, :-num_embeddings]) # df = df.iloc[: , :-1]
test_mean_noEmbeddings = np.array(pd.DataFrame(X_test_mean, columns = columns).iloc[:, :-num_embeddings])
test_median_noEmbeddings = np.array(pd.DataFrame(X_test_median, columns = columns).iloc[:, :-num_embeddings])
test_mean_noEmbeddings_all = np.array(pd.DataFrame(X_test_mean_all, columns = columns).iloc[:, :-num_embeddings])
test_median_noEmbeddings_all = np.array(pd.DataFrame(X_test_median_all, columns = columns).iloc[:, :-num_embeddings])

In [ ]:
missed_data = X_test_mean_all
missed_data = torch.from_numpy(missed_data).float()

train_data = torch.from_numpy(train_data).float()

train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=batch_size,
                                           shuffle=True)

In [ ]:
missed_data_noEm = test_mean_noEmbeddings_all
missed_data_noEm = torch.from_numpy(missed_data_noEm).float()

train_data_noEmbeddings = torch.from_numpy(train_data_noEmbeddings).float()

train_loader_noEm = torch.utils.data.DataLoader(dataset=train_data_noEmbeddings,
                                           batch_size=batch_size,
                                           shuffle=True)

## Autoencoder - without embeddings

In [ ]:
model = Autoencoder(dim=cols-num_embeddings).to(device)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
training(num_epochs, model, train_loader_noEm, criterion, optimizer)

epoch : 1/200, recon loss = 0.01529659
epoch : 2/200, recon loss = 0.01418361
epoch : 3/200, recon loss = 0.01256301
epoch : 4/200, recon loss = 0.00741972
epoch : 5/200, recon loss = 0.00631517
epoch : 6/200, recon loss = 0.00621053
epoch : 7/200, recon loss = 0.00613695
epoch : 8/200, recon loss = 0.00609543
epoch : 9/200, recon loss = 0.00604837
epoch : 10/200, recon loss = 0.00601578
epoch : 11/200, recon loss = 0.00595585
epoch : 12/200, recon loss = 0.00587455
epoch : 13/200, recon loss = 0.00579482
epoch : 14/200, recon loss = 0.00552471
epoch : 15/200, recon loss = 0.00499643
epoch : 16/200, recon loss = 0.00423957
epoch : 17/200, recon loss = 0.00383543
epoch : 18/200, recon loss = 0.00374121
epoch : 19/200, recon loss = 0.00365074
epoch : 20/200, recon loss = 0.00368156
epoch : 21/200, recon loss = 0.00358289
epoch : 22/200, recon loss = 0.00359027
epoch : 23/200, recon loss = 0.00357586
epoch : 24/200, recon loss = 0.00363234
epoch : 25/200, recon loss = 0.00358037
epoch : 2

In [ ]:
model.eval()
rmse_sum = 0

filled_data = model(missed_data_noEm.to(device))
filled_data = filled_data.cpu().detach().numpy()

rmse_sum = rmse_error(test_data, filled_data, cols-num_embeddings, mask)

print(rmse_sum)

1.1560837637956913


## Autoencoder - with embeddings

In [ ]:
model = Autoencoder(dim=cols).to(device)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
training(num_epochs, model, train_loader, criterion, optimizer)

epoch : 1/200, recon loss = 0.02462699
epoch : 2/200, recon loss = 0.02235821
epoch : 3/200, recon loss = 0.01862748
epoch : 4/200, recon loss = 0.01543054
epoch : 5/200, recon loss = 0.01503805
epoch : 6/200, recon loss = 0.01475612
epoch : 7/200, recon loss = 0.01449390
epoch : 8/200, recon loss = 0.01418221
epoch : 9/200, recon loss = 0.01377359
epoch : 10/200, recon loss = 0.01337756
epoch : 11/200, recon loss = 0.01296752
epoch : 12/200, recon loss = 0.01257062
epoch : 13/200, recon loss = 0.01242249
epoch : 14/200, recon loss = 0.01220449
epoch : 15/200, recon loss = 0.01187324
epoch : 16/200, recon loss = 0.01163460
epoch : 17/200, recon loss = 0.01161640
epoch : 18/200, recon loss = 0.01143047
epoch : 19/200, recon loss = 0.01141854
epoch : 20/200, recon loss = 0.01130841
epoch : 21/200, recon loss = 0.01125204
epoch : 22/200, recon loss = 0.01104177
epoch : 23/200, recon loss = 0.01103405
epoch : 24/200, recon loss = 0.01080708
epoch : 25/200, recon loss = 0.01060850
epoch : 2

In [ ]:
model.eval()
rmse_sum = 0

filled_data = model(missed_data.to(device))
filled_data = filled_data.cpu().detach().numpy()

rmse_sum = rmse_error(test_data, filled_data, cols, mask)

print(rmse_sum)

1.1978401246034787


## Mean and Median Imputation

### By category

In [ ]:
imputed = X_test_mean
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

1.5219354232992517


In [ ]:
imputed = X_test_median
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

1.6189507261319402


### Across all foods

In [ ]:
imputed = X_test_mean_all
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

2.0916638036635202


In [ ]:
imputed = X_test_median_all
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

2.178197307340046


# 40 Columns

In [ ]:
data_path = 'datasets/average datasets/byColumnsWithEmbeddings/40_columns_embeddings.csv'
num_embeddings = 10
num_columns = 40

In [ ]:
dataset = pd.read_csv(data_path, index_col=0)
dataset[dataset.notna().all(axis=1)].describe()

,fdc_id,1004,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1002,1093,1079,1003,1009,1102,1265,1266,1264,1315,1316,1314,1094,1097,1137,1146,1300,1299,1404,1267,1273,1323,1167,1271,1304,1166,1175,1263,1165,0,1,2,3,4,5,6,7,8,9
count,1.310500e+04,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.00000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000
mean,4.807517e+05,1.824697,27.659061,14.595597,125.003892,0.384163,55.746662,0.476038,0.077504,0.149979,8.349769,0.472742,0.243665,133.394887,0.350355,1.942243,2.561062,42.944411,0.308332,0.116491,0.065003,0.605730,0.211003,0.023788,14.730179,2.983579,78.72934,12.225029,0.005267,0.005886,0.016719,0.004499,0.003870,0.002480,0.118839,0.002989,0.015767,0.012258,0.008632,0.038133,0.006928,21488.189968,176.402583,0.119704,-0.093912,-0.048493,-0.007831,0.051473,0.023110,0.063160,0.043647
std,2.544018e+05,7.838466,99.924214,45.754175,382.322293,1.055340,153.432660,1.483360,0.264892,0.515558,23.549114,4.416751,0.999601,1736.846193,1.186155,6.583455,9.446549,187.200801,1.605577,0.597369,0.559044,4.440086,1.941342,0.148635,56.459805,11.848737,302.68343,49.040700,0.030796,0.043100,0.223785,0.039142,0.065227,0.015745,0.828604,0.017050,0.106190,0.071481,0.054902,0.974266,0.056734,254447.573253,3958.010969,1.112342,0.694392,0.598293,0.511537,0.453723,0.451006,0.417450,0.426074
min,3.198770e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-139555.939272,-7413.329674,-3.349638,-1.585647,-1.527264,-1.442526,-1.189873,-1.064457,-1.076825,-0.829557
25%,3.256480e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-133692.806088,-2328.507401,-0.796369,-0.667107,-0.477074,-0.343775,-0.236110,-0.286783,-0.177237,-0.292813
50%,3.314100e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-127823.125712,-153.274226,-0.002647,-0.011945,-0.071966,-0.022448,0.012030,0.040891,0.040572,-0.032707
75%,7.474680e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,288262.097477,3032.870912,0.983044,0.441140,0.323270,0.289041,0.257717,0.227563,0.309656,0.274336
max,1.105897e+06,99.400000,1390.000000,386.000000,2520.000000,7.660000,997.000000,12.700000,1.920000,7.870000,96.700000,99.600000,13.200000,40700.000000,11.500000,79.900000,43.900000,1790.000000,22.800000

In [ ]:
data = dataset
train_data, test_data = train_test_split(data, test_size=test_size)

In [ ]:
train_data.describe()

,fdc_id,1004,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1002,1093,1079,1003,1009,1102,1265,1266,1264,1315,1316,1314,1094,1097,1137,1146,1300,1299,1404,1267,1273,1323,1167,1271,1304,1166,1175,1263,1165,0,1,2,3,4,5,6,7,8,9
count,1.048400e+04,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000
mean,4.837058e+05,1.887024,27.318962,14.395221,123.668447,0.380351,55.097482,0.471182,0.076515,0.148187,8.491958,0.488262,0.250330,122.514594,0.342884,1.927489,2.517378,42.457030,0.315906,0.118237,0.069707,0.614797,0.199738,0.024070,14.494945,2.935969,77.261351,11.975630,0.005308,0.006134,0.016559,0.004545,0.003895,0.002414,0.122711,0.003012,0.015654,0.012506,0.008658,0.044606,0.007359,24442.898852,170.589040,0.119443,-0.093123,-0.052680,-0.011590,0.052038,0.022672,0.063588,0.045477
std,2.553333e+05,8.019547,99.862708,45.531646,380.806938,1.051379,152.479658,1.480380,0.263713,0.515968,23.679751,4.518943,1.014003,1618.739582,1.172874,6.563284,9.370949,186.008101,1.641078,0.605666,0.602119,4.534074,1.776330,0.150727,56.099160,11.764307,299.832117,48.511157,0.030991,0.044737,0.216067,0.039463,0.066432,0.015003,0.842898,0.017028,0.104826,0.073419,0.055229,1.087993,0.059635,255379.454754,3933.834962,1.111404,0.692907,0.599123,0.506658,0.454648,0.452795,0.421476,0.425888
min,3.198770e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-139555.939272,-7413.329674,-3.349638,-1.577342,-1.527264,-1.442526,-1.189873,-1.006986,-1.076825,-0.829557
25%,3.256918e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-133648.238488,-2307.662167,-0.798023,-0.663338,-0.477425,-0.344402,-0.236082,-0.286880,-0.177357,-0.292446
50%,3.314685e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-127763.532461,-159.406808,0.006781,-0.011148,-0.072708,-0.023806,0.011936,0.040795,0.041052,-0.032263
75%,7.476032e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,288399.875032,3016.236880,0.981714,0.441011,0.322800,0.288232,0.261384,0.227468,0.311777,0.276468
max,1.105896e+06,99.200000,1380.000000,386.000000,2520.000000,7.660000,997.000000,12.700000,1.920000,7.870000,96.000000,99.600000,13.200000,40700.000000,11.500000,79.900000,43.900000,1730.000000,22.

In [ ]:
test_data.describe()

,fdc_id,1004,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1002,1093,1079,1003,1009,1102,1265,1266,1264,1315,1316,1314,1094,1097,1137,1146,1300,1299,1404,1267,1273,1323,1167,1271,1304,1166,1175,1263,1165,0,1,2,3,4,5,6,7,8,9
count,2.621000e+03,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000
mean,4.689355e+05,1.575387,29.019458,15.397100,130.345670,0.399412,58.343380,0.495464,0.081459,0.157148,7.781011,0.410660,0.217005,176.916063,0.380237,2.001263,2.735799,44.893934,0.278035,0.109507,0.046184,0.569463,0.256064,0.022656,15.671118,3.174018,84.601297,13.222625,0.005103,0.004891,0.017362,0.004317,0.003768,0.002741,0.103353,0.002899,0.016219,0.011265,0.008528,0.012241,0.005205,9669.354429,199.656755,0.120749,-0.097069,-0.031746,0.007202,0.049214,0.024860,0.061447,0.036329
std,2.503407e+05,7.063825,100.177400,46.633877,388.352177,1.071102,157.189722,1.495347,0.269571,0.513949,23.014814,3.982042,0.939498,2145.026530,1.237634,6.664468,9.742956,191.921278,1.454837,0.563017,0.334787,4.042945,2.494442,0.139977,57.881263,12.181039,313.819887,51.101416,0.030006,0.035801,0.252354,0.037836,0.060177,0.018420,0.768730,0.017142,0.111496,0.063141,0.053586,0.101575,0.043194,250385.040634,4053.954334,1.116300,0.700426,0.594781,0.530437,0.450082,0.443857,0.401017,0.426822
min,3.198820e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-139550.845832,-7412.348752,-2.571016,-1.585647,-1.327981,-1.442505,-1.189847,-1.064457,-1.061308,-0.808859
25%,3.254200e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-133925.066958,-2397.174257,-0.788091,-0.671897,-0.475166,-0.334340,-0.238444,-0.265503,-0.170750,-0.294423
50%,3.309040e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-128338.581850,-98.834426,-0.018865,-0.019108,-0.066464,-0.015501,0.012356,0.048059,0.028430,-0.034130
75%,7.469940e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,287779.239352,3128.023695,0.990449,0.441551,0.339117,0.305718,0.247664,0.227888,0.300948,0.264765
max,1.105897e+06,99.400000,1390.000000,358.000000,1810.000000,7.460000,997.000000,9.890000,1.830000,3.900000,96.700000,99.300000,13.000000,40300.000000,11.200000,48.100000,43.200000,1790.000000,22.600000,6.860000,7.100000,69.900000,52.600000,3.32000

In [ ]:
indices = test_data['fdc_id']
indices

1309      322529
3496      325966
8951      335679
12777    1105515
9704      747274
          ...   
4254      327175
4402      327370
8410      335104
5694      329579
5764      329677
Name: fdc_id, Length: 2621, dtype: int64

In [ ]:
del train_data['fdc_id']
del test_data['fdc_id']
columns = train_data.columns

In [ ]:
train_data = train_data.to_numpy()
test_data.fillna(0, inplace=True)
test_data = test_data.to_numpy()
data = dataset.values
rows, cols = data.shape
cols -= 1

y_test = test_data.copy()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
missed_data, missed_data_train, mask = missing_method(test_data, train_data, num_embeddings)
pd.DataFrame(missed_data).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
count,2081.000000,2098.000000,2065.000000,2083.000000,2080.000000,2086.000000,2057.000000,2087.000000,2109.000000,2121.000000,2092.000000,2113.000000,2095.000000,2079.000000,2109.000000,2108.000000,2129.000000,2090.000000,2097.000000,2129.000000,2044.000000,2104.000000,2123.000000,2054.000000,2100.000000,2085.000000,2086.000000,2074.000000,2104.000000,2121.00000,2099.000000,2097.000000,2103.000000,2109.000000,2086.000000,2109.000000,2110.000000,2130.000000,2122.000000,2092.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000
mean,1.625132,29.615825,15.899952,124.521843,0.382135,57.976510,0.516393,0.080287,0.156195,7.633182,0.440550,0.235542,185.649642,0.373641,1.932248,2.718221,44.560263,0.288953,0.101189,0.048570,0.589414,0.248271,0.022463,16.201558,3.297667,88.138609,12.980585,0.005081,0.004599,0.01913,0.004288,0.003748,0.002780,0.106002,0.002992,0.015840,0.011400,0.007671,0.013134,0.005845,9669.354429,199.656755,0.120749,-0.097069,-0.031746,0.007202,0.049214,0.024860,0.061447,0.036329
std,7.297463,101.869652,47.249550,380.291582,1.058164,155.631801,1.532044,0.268996,0.515979,22.701211,4.427868,0.988084,2235.922331,1.221313,6.602178,9.714530,192.164300,1.452063,0.540959,0.345796,4.141566,2.465606,0.140614,58.688585,12.455273,319.828725,50.171650,0.030617,0.034833,0.27946,0.037970,0.058050,0.018405,0.792091,0.017595,0.113064,0.064075,0.048936,0.105973,0.046950,250385.040634,4053.954334,1.116300,0.700426,0.594781,0.530437,0.450082,0.443857,0.401017,0.426822
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-139550.845832,-7412.348752,-2.571016,-1.585647,-1.327981,-1.442505,-1.189847,-1.064457,-1.061308,-0.808859
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-133925.066958,-2397.174257,-0.788091,-0.671897,-0.475166,-0.334340,-0.238444,-0.265503,-0.170750,-0.294423
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-128338.581850,-98.834426,-0.018865,-0.019108,-0.066464,-0.015501,0.012356,0.048059,0.028430,-0.034130
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,287779.239352,3128.023695,0.990449,0.441551,0.339117,0.305718,0.247664,0.227888,0.300948,0.264765
max,99.400000,1390.000000,358.000000,1810.000000,7.460000,995.000000,9.890000,1.830000,3.900000,96.700000,99.300000,13.000000,40300.000000,11.200000,48.100000,42.400000,1790.000000,14.000000,6.860000,7.100000,65.000000,52.600000,3.320000,279.000000,79.000000,1640.000000,331.000000,0.435000,0.765000,7.68000,0.726000,1.790000,0.252000,13.400000,0.194000,1.770000,1.590000,0.787000,2.140000,1.020000,646711.616853,7630.592063,2.647168,

In [ ]:
# traditional methods test sets - impute across all foods
X_train = pd.DataFrame(missed_data_train, columns = columns)
X_test_mean_all = pd.DataFrame(missed_data, columns = columns)
X_test_median_all = pd.DataFrame(missed_data, columns = columns)

X_test_mean_all = impute_traditional_all_foods(X_train, X_test_mean_all, 'mean')
X_test_median_all = impute_traditional_all_foods(X_train, X_test_median_all, 'median')

In [ ]:
# Traditional methods test sets - impute by food category
X_test_mean = pd.DataFrame(missed_data, columns = columns, index=indices)
X_test_median = X_test_mean.copy()
df_mean_cat = df_mean[df_mean['fdc_id'].isin(indices)]
X_test_mean = impute_traditional_by_category(X_test_mean, df_mean_cat, indices)
df_median_cat = df_median[df_median['fdc_id'].isin(indices)]
X_test_median = impute_traditional_by_category(X_test_median, df_median_cat, indices)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)
X_test_mean = scaler.transform(X_test_mean)
X_test_median = scaler.transform(X_test_median)
X_test_mean_all = scaler.transform(X_test_mean_all)
X_test_median_all = scaler.transform(X_test_median_all)

y_test = scaler.transform(y_test)

In [ ]:
# datasets without embeddings for autoencoder
train_data_noEmbeddings = pd.DataFrame(train_data, columns = columns)
train_data_noEmbeddings = np.array(train_data_noEmbeddings.iloc[:, :-num_embeddings]) # df = df.iloc[: , :-1]
test_mean_noEmbeddings = np.array(pd.DataFrame(X_test_mean, columns = columns).iloc[:, :-num_embeddings])
test_median_noEmbeddings = np.array(pd.DataFrame(X_test_median, columns = columns).iloc[:, :-num_embeddings])
test_mean_noEmbeddings_all = np.array(pd.DataFrame(X_test_mean_all, columns = columns).iloc[:, :-num_embeddings])
test_median_noEmbeddings_all = np.array(pd.DataFrame(X_test_median_all, columns = columns).iloc[:, :-num_embeddings])

In [ ]:
missed_data = X_test_mean_all
missed_data = torch.from_numpy(missed_data).float()

train_data = torch.from_numpy(train_data).float()

train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=batch_size,
                                           shuffle=True)

In [ ]:
missed_data_noEm = test_mean_noEmbeddings_all
missed_data_noEm = torch.from_numpy(missed_data_noEm).float()

train_data_noEmbeddings = torch.from_numpy(train_data_noEmbeddings).float()

train_loader_noEm = torch.utils.data.DataLoader(dataset=train_data_noEmbeddings,
                                           batch_size=batch_size,
                                           shuffle=True)

## Autoencoder - without embeddings

In [ ]:
model = Autoencoder(dim=cols-num_embeddings).to(device)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
training(num_epochs, model, train_loader_noEm, criterion, optimizer)

epoch : 1/200, recon loss = 0.01119547
epoch : 2/200, recon loss = 0.00909315
epoch : 3/200, recon loss = 0.00602033
epoch : 4/200, recon loss = 0.00438604
epoch : 5/200, recon loss = 0.00417261
epoch : 6/200, recon loss = 0.00411483
epoch : 7/200, recon loss = 0.00406816
epoch : 8/200, recon loss = 0.00399762
epoch : 9/200, recon loss = 0.00392510
epoch : 10/200, recon loss = 0.00380531
epoch : 11/200, recon loss = 0.00364834
epoch : 12/200, recon loss = 0.00346005
epoch : 13/200, recon loss = 0.00331112
epoch : 14/200, recon loss = 0.00316740
epoch : 15/200, recon loss = 0.00311514
epoch : 16/200, recon loss = 0.00300763
epoch : 17/200, recon loss = 0.00295820
epoch : 18/200, recon loss = 0.00293936
epoch : 19/200, recon loss = 0.00289737
epoch : 20/200, recon loss = 0.00286893
epoch : 21/200, recon loss = 0.00285394
epoch : 22/200, recon loss = 0.00283548
epoch : 23/200, recon loss = 0.00280714
epoch : 24/200, recon loss = 0.00280493
epoch : 25/200, recon loss = 0.00275761
epoch : 2

In [ ]:
model.eval()
rmse_sum = 0

filled_data = model(missed_data_noEm.to(device))
filled_data = filled_data.cpu().detach().numpy()

rmse_sum = rmse_error(test_data, filled_data, cols-num_embeddings, mask)

print(rmse_sum)

1.8480004416035005


## Autoencoder - with embeddings

In [ ]:
model = Autoencoder(dim=cols).to(device)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
training(num_epochs, model, train_loader, criterion, optimizer)

epoch : 1/200, recon loss = 0.01978195
epoch : 2/200, recon loss = 0.01637038
epoch : 3/200, recon loss = 0.01545645
epoch : 4/200, recon loss = 0.01315527
epoch : 5/200, recon loss = 0.01141412
epoch : 6/200, recon loss = 0.01078181
epoch : 7/200, recon loss = 0.01042307
epoch : 8/200, recon loss = 0.01011216
epoch : 9/200, recon loss = 0.00977376
epoch : 10/200, recon loss = 0.00956904
epoch : 11/200, recon loss = 0.00940188
epoch : 12/200, recon loss = 0.00927159
epoch : 13/200, recon loss = 0.00917862
epoch : 14/200, recon loss = 0.00906874
epoch : 15/200, recon loss = 0.00896763
epoch : 16/200, recon loss = 0.00881151
epoch : 17/200, recon loss = 0.00871635
epoch : 18/200, recon loss = 0.00862613
epoch : 19/200, recon loss = 0.00843092
epoch : 20/200, recon loss = 0.00834418
epoch : 21/200, recon loss = 0.00828966
epoch : 22/200, recon loss = 0.00818139
epoch : 23/200, recon loss = 0.00812029
epoch : 24/200, recon loss = 0.00805194
epoch : 25/200, recon loss = 0.00792620
epoch : 2

In [ ]:
model.eval()
rmse_sum = 0

filled_data = model(missed_data.to(device))
filled_data = filled_data.cpu().detach().numpy()

rmse_sum = rmse_error(test_data, filled_data, cols, mask)

print(rmse_sum)

1.8551008859606322


## Mean and Median Imputation

### By category

In [ ]:
imputed = X_test_mean
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

2.531217748333226


In [ ]:
imputed = X_test_median
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

2.680654619816815


### Across all foods

In [ ]:
imputed = X_test_mean_all
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

3.350671086407284


In [ ]:
imputed = X_test_median_all
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

3.4719214819705706


# 80 Columns

In [ ]:
data_path = 'datasets/avarage datasets/byColumnsWithEmbeddings/80_columns_embeddings.csv'
num_embeddings = 20
num_columns = 80

In [ ]:
dataset = pd.read_csv(data_path, index_col=0)
dataset[dataset.notna().all(axis=1)].describe()

,fdc_id,1004,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1002,1093,1079,1003,1009,1102,1265,1266,1264,1315,1316,1314,1094,1097,1137,1146,1300,1299,1404,1267,1273,1323,1167,1271,1304,1166,1175,1263,...,1277,1292,1258,1293,1405,1010,1170,1109,1177,1178,1312,2009,1334,1257,1100,1107,1223,1224,1210,1211,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
count,1.310500e+04,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.00000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,...,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000
mean,4.807517e+05,1.824697,27.659061,14.595597,125.003892,0.384163,55.746662,0.476038,0.077504,0.149979,8.349769,0.472742,0.243665,133.394887,0.350355,1.942243,2.561062,42.944411,0.308332,0.116491,0.065003,0.605730,0.211003,0.023788,14.730179,2.983579,78.72934,12.225029,0.005267,0.005886,0.016719,0.004499,0.003870,0.002480,0.118839,0.002989,0.015767,0.012258,0.008632,0.038133,...,0.002201,0.434562,0.319591,0.150495,0.000279,0.147034,0.011803,0.073772,1.055017,0.027458,0.000093,0.002787,0.000065,0.008114,12.187135,8.910568,0.016636,0.027917,0.002017,0.007339,21488.189968,176.402583,0.119704,-0.093912,-0.048494,-0.007831,0.051473,0.023113,0.063163,0.043670,-0.005261,-0.053342,0.032862,0.017149,0.053913,-0.050025,-0.006816,-0.001294,0.045430,-0.002778
std,2.544018e+05,7.838466,99.924214,45.754175,382.322293,1.055340,153.432660,1.483360,0.264892,0.515558,23.549114,4.416751,0.999601,1736.846193,1.186155,6.583455,9.446549,187.200801,1.605577,0.597369,0.559044,4.440086,1.941342,0.148635,56.459805,11.848737,302.68343,49.040700,0.030796,0.043100,0.223785,0.039142,0.065227,0.015745,0.828604,0.017050,0.106190,0.071481,0.054902,0.974266,...,0.025626,4.338678,2.546196,1.960438,0.002974,2.828219,0.085062,1.032461,10.123472,0.206489,0.001497,0.029047,0.000516,0.083699,237.511065,175.426202,0.181747,0.318716,0.022631,0.078590,254447.573253,3958.010969,1.112342,0.694392,0.598293,0.511537,0.453723,0.451007,0.417454,0.426071,0.400175,0.345475,0.349559,0.332055,0.321275,0.292891,0.270026,0.284946,0.261710,0.269164
min,3.198770e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-139555.939272,-7413.329674,-3.349638,-1.585647,-1.527263,-1.442526,-1.189856,-1.064453,-1.077035,-0.829624,-0.933852,-0.969973,-1.097073,-0.920129,-1.006213,-1.000462,-0.798443,-0.685798,-0.669546,-0.809746
25%,3.256480e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [ ]:
data = dataset
train_data, test_data = train_test_split(data, test_size=test_size)

In [ ]:
train_data.describe()

,fdc_id,1004,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1002,1093,1079,1003,1009,1102,1265,1266,1264,1315,1316,1314,1094,1097,1137,1146,1300,1299,1404,1267,1273,1323,1167,1271,1304,1166,1175,1263,...,1277,1292,1258,1293,1405,1010,1170,1109,1177,1178,1312,2009,1334,1257,1100,1107,1223,1224,1210,1211,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
count,1.048400e+04,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,...,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000
mean,4.808406e+05,1.788952,27.818199,14.610349,125.796356,0.381077,55.589088,0.473794,0.077637,0.150204,8.280277,0.466951,0.235234,130.758585,0.355857,1.938891,2.572338,42.944601,0.312186,0.116781,0.064787,0.635888,0.215604,0.024394,14.746471,2.996175,78.773655,12.361551,0.005233,0.005803,0.015959,0.004814,0.004390,0.002434,0.120633,0.002984,0.015861,0.012295,0.008884,0.039484,...,0.002451,0.420075,0.306654,0.147405,0.000279,0.165155,0.011789,0.078353,0.988936,0.027461,0.000099,0.002724,0.000066,0.008259,11.711017,8.692865,0.016277,0.027779,0.001996,0.007305,21577.236256,166.880759,0.116692,-0.094338,-0.047668,-0.012898,0.056047,0.022257,0.061427,0.043563,-0.005482,-0.054468,0.031648,0.018106,0.054145,-0.051153,-0.005925,0.000058,0.046189,-0.002405
std,2.545874e+05,7.742546,100.368856,45.772164,383.926355,1.048002,153.150946,1.474350,0.264757,0.516615,23.518922,4.413584,0.981380,1700.110237,1.198870,6.577799,9.472052,187.192828,1.609271,0.593455,0.563034,4.639686,1.932338,0.155189,56.453297,11.869546,302.284160,49.438662,0.030464,0.042131,0.208596,0.041514,0.071241,0.015491,0.851130,0.017098,0.106631,0.070982,0.055063,1.007735,...,0.027812,4.227437,2.382598,1.910046,0.003049,3.143208,0.085169,1.058159,9.861488,0.208564,0.001648,0.028177,0.000529,0.084488,230.035809,177.214654,0.177415,0.311426,0.022198,0.077407,254633.088377,3965.821318,1.112802,0.695036,0.597310,0.509655,0.454499,0.449308,0.418755,0.426148,0.401053,0.343192,0.350124,0.332365,0.321075,0.293762,0.269050,0.286023,0.261000,0.270059
min,3.198770e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-139555.939272,-7413.329674,-3.349638,-1.585647,-1.527263,-1.442526,-1.189856,-1.064453,-1.077035,-0.829624,-0.933852,-0.969973,-1.097073,-0.920129,-1.006163,-1.000462,-0.798443,-0.685798,-0.669546,-0.809746
25%,3.256138e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [ ]:
test_data.describe()

,fdc_id,1004,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1002,1093,1079,1003,1009,1102,1265,1266,1264,1315,1316,1314,1094,1097,1137,1146,1300,1299,1404,1267,1273,1323,1167,1271,1304,1166,1175,1263,...,1277,1292,1258,1293,1405,1010,1170,1109,1177,1178,1312,2009,1334,1257,1100,1107,1223,1224,1210,1211,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
count,2.621000e+03,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,...,2621.000000,2621.000000,2621.00000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000
mean,4.803963e+05,1.967676,27.022510,14.536589,121.834033,0.396509,56.376955,0.485013,0.076968,0.149080,8.627738,0.495906,0.277390,143.940099,0.328348,1.955654,2.515960,42.943647,0.292919,0.115330,0.065867,0.485100,0.192602,0.021364,14.665013,2.933193,78.552079,11.678939,0.005403,0.006216,0.019760,0.003240,0.001788,0.002662,0.111663,0.003010,0.015391,0.012110,0.007625,0.032726,...,0.001202,0.492513,0.37134,0.162857,0.000279,0.074552,0.011859,0.055448,1.319344,0.027448,0.000068,0.003040,0.000063,0.007536,14.091606,9.781381,0.018074,0.028465,0.002101,0.007475,21132.004815,214.489880,0.131754,-0.092210,-0.051795,0.012438,0.033177,0.026538,0.070110,0.044096,-0.004377,-0.048841,0.037719,0.013325,0.052985,-0.045513,-0.010380,-0.006703,0.042393,-0.004267
std,2.537059e+05,8.210923,98.141815,45.690826,375.893218,1.084308,154.582160,1.519119,0.265481,0.511403,23.671963,4.430164,1.068915,1876.920647,1.133821,6.607275,9.345496,187.268411,1.590927,0.612892,0.542897,3.528785,1.977214,0.118853,56.496558,11.767255,304.333119,47.420738,0.032094,0.046780,0.276343,0.027657,0.031095,0.016725,0.731701,0.016862,0.104423,0.073456,0.054255,0.827105,...,0.013715,4.758129,3.11585,2.150577,0.002654,0.685144,0.084649,0.922500,11.107897,0.198013,0.000584,0.032298,0.000462,0.080481,265.353791,168.112176,0.198158,0.346407,0.024288,0.083169,253752.326858,3927.135566,1.110629,0.691941,0.602314,0.518598,0.450228,0.457808,0.412216,0.425847,0.396720,0.354490,0.347314,0.330849,0.322132,0.289392,0.273918,0.280586,0.264559,0.265600
min,3.198780e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-139554.920584,-7410.386832,-2.489668,-1.577342,-1.433074,-1.442400,-1.189727,-0.922758,-1.021707,-0.808912,-0.933827,-0.946624,-0.961907,-0.890940,-1.006213,-0.894695,-0.797834,-0.601732,-0.616257,-0.809725
25%,3.257840e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000

In [ ]:
indices = test_data['fdc_id']
indices

9313      336041
2440      324243
2257      323937
1041      322215
12468    1105140
          ...   
11687     790512
3464      325917
11720     790553
2688      324702
5900      329898
Name: fdc_id, Length: 2621, dtype: int64

In [ ]:
del train_data['fdc_id']
del test_data['fdc_id']
columns = train_data.columns

In [ ]:
train_data = train_data.to_numpy()
test_data.fillna(0, inplace=True)
test_data = test_data.to_numpy()
data = dataset.values
rows, cols = data.shape
cols -= 1

y_test = test_data.copy()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
missed_data, missed_data_train, mask = missing_method(test_data, train_data, num_embeddings)
pd.DataFrame(missed_data).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
count,2109.000000,2078.000000,2092.00000,2100.000000,2123.00000,2085.000000,2137.000000,2119.000000,2110.000000,2086.000000,2088.000000,2079.000000,2117.000000,2126.000000,2125.000000,2110.000000,2120.000000,2138.000000,2114.000000,2104.000000,2088.000000,2118.000000,2109.000000,2123.00000,2083.000000,2091.000000,2100.000000,2140.000000,2087.000000,2087.000000,2097.000000,2130.000000,2107.000000,2058.000000,2122.000000,2120.000000,2065.000000,2095.000000,2091.000000,2093.000000,...,2076.000000,2107.000000,2101.000000,2092.000000,2125.000000,2102.000000,2086.000000,2085.000000,2105.000000,2094.000000,2108.00000,2133.000000,2068.000000,2096.000000,2046.000000,2067.000000,2111.000000,2099.000000,2094.000000,2092.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000
mean,1.967435,26.703080,13.71305,120.066667,0.39764,58.003357,0.493776,0.080866,0.152113,8.641716,0.500747,0.254367,161.042985,0.323612,1.978193,2.486318,43.015943,0.297366,0.109253,0.058020,0.437340,0.183017,0.021095,14.16910,3.165098,73.178384,11.415000,0.005707,0.005720,0.021977,0.003471,0.002084,0.002667,0.103992,0.003213,0.014764,0.011882,0.008267,0.037143,0.006607,...,0.001267,0.508391,0.367177,0.180243,0.000321,0.071613,0.011859,0.054974,1.316390,0.025907,0.00008,0.003531,0.000056,0.009091,15.702639,7.275278,0.017606,0.033414,0.001969,0.007199,21132.004815,214.489880,0.131754,-0.092210,-0.051795,0.012438,0.033177,0.026538,0.070110,0.044096,-0.004377,-0.048841,0.037719,0.013325,0.052985,-0.045513,-0.010380,-0.006703,0.042393,-0.004267
std,8.333341,97.186923,43.91800,372.767341,1.09067,155.870192,1.527220,0.272309,0.514505,23.644255,4.442314,0.983221,2083.637258,1.131511,6.652319,9.294418,185.051257,1.647440,0.594519,0.426314,3.169038,1.860204,0.115439,55.54696,12.216444,294.649725,45.966284,0.032902,0.045211,0.307572,0.029392,0.034465,0.016328,0.673840,0.017801,0.097919,0.069045,0.057652,0.923418,0.058450,...,0.015056,4.994738,3.059325,2.373638,0.002881,0.651726,0.083499,0.908443,11.203809,0.186068,0.00064,0.035284,0.000442,0.089476,285.237165,158.288625,0.207159,0.383715,0.021935,0.081271,253752.326858,3927.135566,1.110629,0.691941,0.602314,0.518598,0.450228,0.457808,0.412216,0.425847,0.396720,0.354490,0.347314,0.330849,0.322132,0.289392,0.273918,0.280586,0.264559,0.265600
min,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-139554.920584,-7410.386832,-2.489668,-1.577342,-1.433074,-1.442400,-1.189727,-0.922758,-1.021707,-0.808912,-0.933827,-0.946624,-0.961907,-0.890940,-1.006213,-0.894695,-0.797834,-0.601732,-0.616257,-0.809725
25%,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [ ]:
# traditional methods test sets - impute across all foods
X_train = pd.DataFrame(missed_data_train, columns = columns)
X_test_mean_all = pd.DataFrame(missed_data, columns = columns)
X_test_median_all = pd.DataFrame(missed_data, columns = columns)

X_test_mean_all = impute_traditional_all_foods(X_train, X_test_mean_all, 'mean')
X_test_median_all = impute_traditional_all_foods(X_train, X_test_median_all, 'median')

In [ ]:
# Traditional methods test sets - impute by food category
X_test_mean = pd.DataFrame(missed_data, columns = columns, index=indices)
X_test_median = X_test_mean.copy()
df_mean_cat = df_mean[df_mean['fdc_id'].isin(indices)]
X_test_mean = impute_traditional_by_category(X_test_mean, df_mean_cat, indices)
df_median_cat = df_median[df_median['fdc_id'].isin(indices)]
X_test_median = impute_traditional_by_category(X_test_median, df_median_cat, indices)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)
X_test_mean = scaler.transform(X_test_mean)
X_test_median = scaler.transform(X_test_median)
X_test_mean_all = scaler.transform(X_test_mean_all)
X_test_median_all = scaler.transform(X_test_median_all)

y_test = scaler.transform(y_test)

In [ ]:
# datasets without embeddings for autoencoder
train_data_noEmbeddings = pd.DataFrame(train_data, columns = columns)
train_data_noEmbeddings = np.array(train_data_noEmbeddings.iloc[:, :-num_embeddings]) # df = df.iloc[: , :-1]
test_mean_noEmbeddings = np.array(pd.DataFrame(X_test_mean, columns = columns).iloc[:, :-num_embeddings])
test_median_noEmbeddings = np.array(pd.DataFrame(X_test_median, columns = columns).iloc[:, :-num_embeddings])
test_mean_noEmbeddings_all = np.array(pd.DataFrame(X_test_mean_all, columns = columns).iloc[:, :-num_embeddings])
test_median_noEmbeddings_all = np.array(pd.DataFrame(X_test_median_all, columns = columns).iloc[:, :-num_embeddings])

In [ ]:
missed_data = X_test_mean_all
missed_data = torch.from_numpy(missed_data).float()

train_data = torch.from_numpy(train_data).float()

train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=batch_size,
                                           shuffle=True)

In [ ]:
missed_data_noEm = test_mean_noEmbeddings_all
missed_data_noEm = torch.from_numpy(missed_data_noEm).float()

train_data_noEmbeddings = torch.from_numpy(train_data_noEmbeddings).float()

train_loader_noEm = torch.utils.data.DataLoader(dataset=train_data_noEmbeddings,
                                           batch_size=batch_size,
                                           shuffle=True)

## Autoencoder - without embeddings

In [ ]:
model = Autoencoder(dim=cols-num_embeddings).to(device)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
training(num_epochs, model, train_loader_noEm, criterion, optimizer)

epoch : 1/200, recon loss = 0.00723645
epoch : 2/200, recon loss = 0.00623200
epoch : 3/200, recon loss = 0.00608631
epoch : 4/200, recon loss = 0.00575263
epoch : 5/200, recon loss = 0.00497927
epoch : 6/200, recon loss = 0.00397156
epoch : 7/200, recon loss = 0.00338542
epoch : 8/200, recon loss = 0.00317286
epoch : 9/200, recon loss = 0.00310097
epoch : 10/200, recon loss = 0.00307003
epoch : 11/200, recon loss = 0.00305066
epoch : 12/200, recon loss = 0.00301768
epoch : 13/200, recon loss = 0.00299938
epoch : 14/200, recon loss = 0.00296950
epoch : 15/200, recon loss = 0.00292809
epoch : 16/200, recon loss = 0.00288915
epoch : 17/200, recon loss = 0.00283167
epoch : 18/200, recon loss = 0.00278632
epoch : 19/200, recon loss = 0.00272362
epoch : 20/200, recon loss = 0.00268940
epoch : 21/200, recon loss = 0.00263934
epoch : 22/200, recon loss = 0.00258300
epoch : 23/200, recon loss = 0.00256760
epoch : 24/200, recon loss = 0.00254809
epoch : 25/200, recon loss = 0.00252104
epoch : 2

In [ ]:
model.eval()
rmse_sum = 0

filled_data = model(missed_data_noEm.to(device))
filled_data = filled_data.cpu().detach().numpy()

rmse_sum = rmse_error(test_data, filled_data, cols-num_embeddings, mask)

print(rmse_sum)

3.1464000386280997


## Autoencoder - with embeddings

In [ ]:
model = Autoencoder(dim=cols).to(device)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
training(num_epochs, model, train_loader, criterion, optimizer)

epoch : 1/200, recon loss = 0.01493986
epoch : 2/200, recon loss = 0.01184460
epoch : 3/200, recon loss = 0.01175992
epoch : 4/200, recon loss = 0.01161368
epoch : 5/200, recon loss = 0.01129655
epoch : 6/200, recon loss = 0.01056569
epoch : 7/200, recon loss = 0.00956796
epoch : 8/200, recon loss = 0.00903610
epoch : 9/200, recon loss = 0.00883364
epoch : 10/200, recon loss = 0.00874422
epoch : 11/200, recon loss = 0.00869116
epoch : 12/200, recon loss = 0.00863454
epoch : 13/200, recon loss = 0.00858350
epoch : 14/200, recon loss = 0.00852487
epoch : 15/200, recon loss = 0.00844616
epoch : 16/200, recon loss = 0.00835073
epoch : 17/200, recon loss = 0.00825278
epoch : 18/200, recon loss = 0.00815671
epoch : 19/200, recon loss = 0.00804786
epoch : 20/200, recon loss = 0.00797374
epoch : 21/200, recon loss = 0.00788826
epoch : 22/200, recon loss = 0.00784343
epoch : 23/200, recon loss = 0.00779354
epoch : 24/200, recon loss = 0.00773754
epoch : 25/200, recon loss = 0.00769605
epoch : 2

In [ ]:
model.eval()
rmse_sum = 0

filled_data = model(missed_data.to(device))
filled_data = filled_data.cpu().detach().numpy()

rmse_sum = rmse_error(test_data, filled_data, cols, mask)

print(rmse_sum)

3.1930650168502


## Mean and Median Imputation

### By category

In [ ]:
imputed = X_test_mean
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

4.240093997588478


In [ ]:
imputed = X_test_median
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

4.456391764474138


### Across all foods

In [ ]:
imputed = X_test_mean_all
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

5.005264371707832


In [ ]:
imputed = X_test_median_all
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

5.1325066538984725


# 120 Columns

In [ ]:
data_path = 'datasets/average datasets/byColumnsWithEmbeddings/120_columns_embeddings.csv'
num_embeddings = 30
num_columns = 120

In [ ]:
dataset = pd.read_csv(data_path, index_col=0)
dataset[dataset.notna().all(axis=1)].describe()

,fdc_id,1004,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1002,1093,1079,1003,1009,1102,1265,1266,1264,1315,1316,1314,1094,1097,1137,1146,1300,1299,1404,1267,1273,1323,1167,1271,1304,1166,1175,1263,...,1180,1194,1128,1195,1005,1075,1196,1197,1198,1084,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
count,1.310500e+04,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.00000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,...,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000
mean,4.807517e+05,1.824697,27.659061,14.595597,125.003892,0.384163,55.746662,0.476038,0.077504,0.149979,8.349769,0.472742,0.243665,133.394887,0.350355,1.942243,2.561062,42.944411,0.308332,0.116491,0.065003,0.605730,0.211003,0.023788,14.730179,2.983579,78.72934,12.225029,0.005267,0.005886,0.016719,0.004499,0.003870,0.002480,0.118839,0.002989,0.015767,0.012258,0.008632,0.038133,...,0.928287,0.039206,0.002494,0.726326,0.164999,0.002459,0.045410,0.042976,0.046929,0.021892,21488.189968,176.402583,0.119704,-0.093912,-0.048494,-0.007831,0.051473,0.023113,0.063163,0.043670,-0.005261,-0.053342,0.032862,0.017148,0.053914,-0.050016,-0.006804,-0.001309,0.045397,-0.002817,-0.004870,0.015158,-0.002832,-0.008018,0.007639,-0.005292,0.013278,0.026410,-0.008193,-0.003744
std,2.544018e+05,7.838466,99.924214,45.754175,382.322293,1.055340,153.432660,1.483360,0.264892,0.515558,23.549114,4.416751,0.999601,1736.846193,1.186155,6.583455,9.446549,187.200801,1.605577,0.597369,0.559044,4.440086,1.941342,0.148635,56.459805,11.848737,302.68343,49.040700,0.030796,0.043100,0.223785,0.039142,0.065227,0.015745,0.828604,0.017050,0.106190,0.071481,0.054902,0.974266,...,15.719211,0.824407,0.055645,14.277867,2.905073,0.034625,1.333663,0.624933,1.671006,0.319893,254447.573253,3958.010969,1.112342,0.694392,0.598293,0.511537,0.453723,0.451007,0.417454,0.426072,0.400175,0.345475,0.349559,0.332055,0.321282,0.292925,0.270047,0.284938,0.261697,0.269206,0.240664,0.243863,0.245151,0.247991,0.235566,0.205593,0.224944,0.212094,0.209662,0.206100
min,3.198770e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-139555.939272,-7413.329674,-3.349638,-1.585647,-1.527263,-1.442526,-1.189856,-1.064453,-1.077035,-0.829626,-0.933856,-0.969972,-1.097135,-0.920029,-1.006142,-1.000203,-0.798836,-0.687075,-0.668804,-0.801703,-0.726147,-0.698734,-0.733370,-0.848113,-0.673804,-0.609317,-0.622568,-0.618201,-0.605566,-0.802281
25%,3.256480e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [ ]:
data = dataset
train_data, test_data = train_test_split(data, test_size=test_size)

In [ ]:
train_data.describe()

,fdc_id,1004,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1002,1093,1079,1003,1009,1102,1265,1266,1264,1315,1316,1314,1094,1097,1137,1146,1300,1299,1404,1267,1273,1323,1167,1271,1304,1166,1175,1263,...,1180,1194,1128,1195,1005,1075,1196,1197,1198,1084,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
count,1.048400e+04,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,...,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000
mean,4.807307e+05,1.838933,27.191816,14.418333,124.497043,0.377420,55.049886,0.472891,0.076689,0.147316,8.377778,0.465995,0.247112,126.185712,0.347301,1.912456,2.568743,42.085340,0.315506,0.119174,0.067715,0.623412,0.223398,0.024106,14.656143,2.980733,78.750095,12.213277,0.005360,0.005941,0.017104,0.004821,0.004366,0.002521,0.119267,0.003027,0.015955,0.011941,0.008555,0.043831,...,0.865958,0.035702,0.002479,0.663859,0.168130,0.002428,0.054016,0.043018,0.052289,0.022568,21467.008441,182.062106,0.119423,-0.095347,-0.048735,-0.007134,0.054944,0.023179,0.063675,0.043259,-0.004838,-0.051300,0.030691,0.017925,0.052880,-0.050042,-0.004916,-0.001487,0.045954,-0.004037,-0.004376,0.015646,-0.001045,-0.007970,0.006989,-0.004153,0.012878,0.027332,-0.007871,-0.002815
std,2.543947e+05,7.873509,98.818185,45.429656,382.049756,1.047533,152.629159,1.481276,0.263418,0.507523,23.608917,4.309402,1.010590,1662.616839,1.178584,6.533157,9.457161,184.404872,1.624915,0.606916,0.593610,4.496240,2.050528,0.151031,56.189047,11.840536,302.770335,49.054157,0.031136,0.043461,0.226249,0.041357,0.070544,0.015894,0.836070,0.017135,0.106890,0.071769,0.054803,1.087551,...,15.102529,0.693033,0.056443,13.714635,2.924708,0.035324,1.488693,0.619556,1.854747,0.329757,254440.586435,3952.715216,1.109124,0.693789,0.599498,0.508734,0.453486,0.450050,0.417313,0.425748,0.400349,0.344332,0.351216,0.333003,0.321272,0.293894,0.270204,0.283863,0.263103,0.268481,0.241821,0.243146,0.245953,0.249616,0.235283,0.205396,0.225057,0.212131,0.210375,0.207533
min,3.198770e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-139555.939272,-7413.329674,-3.349638,-1.585647,-1.482007,-1.442526,-1.189856,-1.064453,-1.077035,-0.829626,-0.933843,-0.969972,-1.097135,-0.920029,-1.006142,-1.000139,-0.798836,-0.661197,-0.668804,-0.801681,-0.726147,-0.698734,-0.733370,-0.848113,-0.673804,-0.609317,-0.622568,-0.618201,-0.605566,-0.802281
25%,3.256368e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [ ]:
test_data.describe()

,fdc_id,1004,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1002,1093,1079,1003,1009,1102,1265,1266,1264,1315,1316,1314,1094,1097,1137,1146,1300,1299,1404,1267,1273,1323,1167,1271,1304,1166,1175,1263,...,1180,1194,1128,1195,1005,1075,1196,1197,1198,1084,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
count,2.621000e+03,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,...,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000
mean,4.808360e+05,1.767753,29.528043,15.304655,127.031286,0.411137,58.533766,0.488626,0.080763,0.160633,8.237734,0.499729,0.229878,162.231591,0.362572,2.061393,2.530340,46.380694,0.279637,0.105758,0.054155,0.535003,0.161425,0.022514,15.026326,2.994964,78.646318,12.272034,0.004897,0.005666,0.015179,0.003214,0.001885,0.002314,0.117130,0.002836,0.015014,0.013524,0.008942,0.015341,...,1.177604,0.053224,0.002552,0.976192,0.152476,0.002579,0.010988,0.042808,0.025486,0.019191,21572.916075,153.764490,0.120828,-0.088173,-0.047529,-0.010620,0.037588,0.022852,0.061117,0.045312,-0.006952,-0.061510,0.041546,0.014043,0.058051,-0.049911,-0.014357,-0.000596,0.043168,0.002061,-0.006847,0.013206,-0.009980,-0.008207,0.010237,-0.009849,0.014877,0.022725,-0.009483,-0.007460
std,2.544785e+05,7.697916,104.229568,47.032037,383.476861,1.085789,156.604018,1.491882,0.270735,0.546488,23.312503,4.823097,0.954442,2006.587891,1.216119,6.780884,9.405717,197.989572,1.525738,0.557532,0.391267,4.208065,1.422409,0.138656,57.540020,11.883744,302.393274,48.996164,0.029397,0.041630,0.213682,0.028586,0.036916,0.015139,0.798193,0.016710,0.103358,0.070314,0.055309,0.119789,...,17.976672,1.215442,0.052341,16.338307,2.825680,0.031680,0.164785,0.646115,0.447997,0.276983,254524.057706,3979.800284,1.125335,0.696901,0.593561,0.522688,0.454492,0.454899,0.418094,0.427441,0.399548,0.349958,0.342780,0.328279,0.321352,0.289075,0.269336,0.289248,0.256030,0.272083,0.236014,0.246748,0.241832,0.241427,0.236721,0.206354,0.224524,0.211945,0.206822,0.200258
min,3.198780e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-139554.920584,-7401.558240,-2.962405,-1.577342,-1.527263,-1.442463,-1.189817,-1.006999,-0.950851,-0.808687,-0.933856,-0.969439,-0.961805,-0.889532,-1.006043,-1.000203,-0.798091,-0.687075,-0.613786,-0.801703,-0.678342,-0.648339,-0.733317,-0.847795,-0.570389,-0.609083,-0.548479,-0.617991,-0.605527,-0.723781
25%,3.257300e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000

In [ ]:
indices = test_data['fdc_id']
indices

10512     748595
12618    1105328
5891      329882
9742      747334
8399      335084
          ...   
12133    1104708
7710      333356
5455      329214
6016      330136
8229      334695
Name: fdc_id, Length: 2621, dtype: int64

In [ ]:
del train_data['fdc_id']
del test_data['fdc_id']
columns = train_data.columns

In [ ]:
train_data = train_data.to_numpy()
test_data.fillna(0, inplace=True)
test_data = test_data.to_numpy()
data = dataset.values
rows, cols = data.shape
cols -= 1

y_test = test_data.copy()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
missed_data, missed_data_train, mask = missing_method(test_data, train_data, num_embeddings)
pd.DataFrame(missed_data).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
count,2079.000000,2131.000000,2122.000000,2116.000000,2114.000000,2112.000000,2101.000000,2086.000000,2104.000000,2071.000000,2086.000000,2115.000000,2104.000000,2096.000000,2098.000000,2087.000000,2097.000000,2102.000000,2109.000000,2098.000000,2086.000000,2053.000000,2091.000000,2081.000000,2129.000000,2076.000000,2088.000000,2074.000000,2119.000000,2121.000000,2093.000000,2129.000000,2108.000000,2122.000000,2104.000000,2108.000000,2109.000000,2072.000000,2064.000000,2115.000000,...,2095.000000,2063.000000,2100.000000,2082.000000,2096.000000,2084.000000,2116.000000,2135.000000,2073.000000,2099.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000
mean,1.822968,29.281089,15.401979,127.583176,0.413018,58.667614,0.482922,0.082035,0.162047,8.100990,0.582287,0.235206,111.011882,0.379628,2.164971,2.341284,49.029375,0.243778,0.105295,0.046996,0.452872,0.150144,0.024538,13.952427,3.262001,75.279865,12.383621,0.005118,0.005440,0.013107,0.003535,0.001947,0.002442,0.121552,0.002490,0.015886,0.014331,0.008495,0.014514,0.007035,...,1.207351,0.056714,0.002152,0.855187,0.133941,0.002893,0.010491,0.040094,0.018186,0.016532,21572.916075,153.764490,0.120828,-0.088173,-0.047529,-0.010620,0.037588,0.022852,0.061117,0.045312,-0.006952,-0.061510,0.041546,0.014043,0.058051,-0.049911,-0.014357,-0.000596,0.043168,0.002061,-0.006847,0.013206,-0.009980,-0.008207,0.010237,-0.009849,0.014877,0.022725,-0.009483,-0.007460
std,7.688367,105.779651,46.881167,386.194247,1.093257,156.451379,1.484678,0.272904,0.555224,23.085199,5.388674,0.954813,1455.138966,1.238272,6.922343,9.048164,203.083355,1.361023,0.549890,0.371828,3.720943,1.443742,0.150145,55.487474,12.358336,295.218949,48.990889,0.030959,0.041524,0.183330,0.030463,0.040624,0.015608,0.825986,0.015069,0.107239,0.072712,0.054448,0.119956,0.052296,...,18.558856,1.345060,0.045922,15.254636,2.571180,0.034639,0.128727,0.617660,0.376355,0.235400,254524.057706,3979.800284,1.125335,0.696901,0.593561,0.522688,0.454492,0.454899,0.418094,0.427441,0.399548,0.349958,0.342780,0.328279,0.321352,0.289075,0.269336,0.289248,0.256030,0.272083,0.236014,0.246748,0.241832,0.241427,0.236721,0.206354,0.224524,0.211945,0.206822,0.200258
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-139554.920584,-7401.558240,-2.962405,-1.577342,-1.527263,-1.442463,-1.189817,-1.006999,-0.950851,-0.808687,-0.933856,-0.969439,-0.961805,-0.889532,-1.006043,-1.000203,-0.798091,-0.687075,-0.613786,-0.801703,-0.678342,-0.648339,-0.733317,-0.847795,-0.570389,-0.609083,-0.548479,-0.617991,-0.605527,-0.723781
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-133609.273672,

In [ ]:
# traditional methods test sets - impute across all foods
X_train = pd.DataFrame(missed_data_train, columns = columns)
X_test_mean_all = pd.DataFrame(missed_data, columns = columns)
X_test_median_all = pd.DataFrame(missed_data, columns = columns)

X_test_mean_all = impute_traditional_all_foods(X_train, X_test_mean_all, 'mean')
X_test_median_all = impute_traditional_all_foods(X_train, X_test_median_all, 'median')

In [ ]:
# Traditional methods test sets - impute by food category
X_test_mean = pd.DataFrame(missed_data, columns = columns, index=indices)
X_test_median = X_test_mean.copy()
df_mean_cat = df_mean[df_mean['fdc_id'].isin(indices)]
X_test_mean = impute_traditional_by_category(X_test_mean, df_mean_cat, indices)
df_median_cat = df_median[df_median['fdc_id'].isin(indices)]
X_test_median = impute_traditional_by_category(X_test_median, df_median_cat, indices)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)
X_test_mean = scaler.transform(X_test_mean)
X_test_median = scaler.transform(X_test_median)
X_test_mean_all = scaler.transform(X_test_mean_all)
X_test_median_all = scaler.transform(X_test_median_all)

y_test = scaler.transform(y_test)

In [ ]:
# datasets without embeddings for autoencoder
train_data_noEmbeddings = pd.DataFrame(train_data, columns = columns)
train_data_noEmbeddings = np.array(train_data_noEmbeddings.iloc[:, :-num_embeddings]) # df = df.iloc[: , :-1]
test_mean_noEmbeddings = np.array(pd.DataFrame(X_test_mean, columns = columns).iloc[:, :-num_embeddings])
test_median_noEmbeddings = np.array(pd.DataFrame(X_test_median, columns = columns).iloc[:, :-num_embeddings])
test_mean_noEmbeddings_all = np.array(pd.DataFrame(X_test_mean_all, columns = columns).iloc[:, :-num_embeddings])
test_median_noEmbeddings_all = np.array(pd.DataFrame(X_test_median_all, columns = columns).iloc[:, :-num_embeddings])

In [ ]:
missed_data = X_test_mean_all
missed_data = torch.from_numpy(missed_data).float()

train_data = torch.from_numpy(train_data).float()

train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=batch_size,
                                           shuffle=True)

In [ ]:
missed_data_noEm = test_mean_noEmbeddings_all
missed_data_noEm = torch.from_numpy(missed_data_noEm).float()

train_data_noEmbeddings = torch.from_numpy(train_data_noEmbeddings).float()

train_loader_noEm = torch.utils.data.DataLoader(dataset=train_data_noEmbeddings,
                                           batch_size=batch_size,
                                           shuffle=True)

## Autoencoder - without embeddings

In [ ]:
model = Autoencoder(dim=cols-num_embeddings).to(device)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
training(num_epochs, model, train_loader_noEm, criterion, optimizer)

epoch : 1/200, recon loss = 0.00581219
epoch : 2/200, recon loss = 0.00472384
epoch : 3/200, recon loss = 0.00469371
epoch : 4/200, recon loss = 0.00465589
epoch : 5/200, recon loss = 0.00459831
epoch : 6/200, recon loss = 0.00449245
epoch : 7/200, recon loss = 0.00428894
epoch : 8/200, recon loss = 0.00391550
epoch : 9/200, recon loss = 0.00337470
epoch : 10/200, recon loss = 0.00294085
epoch : 11/200, recon loss = 0.00269202
epoch : 12/200, recon loss = 0.00257523
epoch : 13/200, recon loss = 0.00252215
epoch : 14/200, recon loss = 0.00249634
epoch : 15/200, recon loss = 0.00247268
epoch : 16/200, recon loss = 0.00246725
epoch : 17/200, recon loss = 0.00245641
epoch : 18/200, recon loss = 0.00244636
epoch : 19/200, recon loss = 0.00243947
epoch : 20/200, recon loss = 0.00242779
epoch : 21/200, recon loss = 0.00241965
epoch : 22/200, recon loss = 0.00240268
epoch : 23/200, recon loss = 0.00238925
epoch : 24/200, recon loss = 0.00236624
epoch : 25/200, recon loss = 0.00235028
epoch : 2

In [ ]:
model.eval()
rmse_sum = 0

filled_data = model(missed_data_noEm.to(device))
filled_data = filled_data.cpu().detach().numpy()

rmse_sum = rmse_error(test_data, filled_data, cols-num_embeddings, mask)

print(rmse_sum)

4.011076875370579


## Autoencoder - with embeddings

In [ ]:
model = Autoencoder(dim=cols).to(device)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
training(num_epochs, model, train_loader, criterion, optimizer)

epoch : 1/200, recon loss = 0.01557843
epoch : 2/200, recon loss = 0.00973100
epoch : 3/200, recon loss = 0.00969905
epoch : 4/200, recon loss = 0.00966056
epoch : 5/200, recon loss = 0.00960115
epoch : 6/200, recon loss = 0.00950456
epoch : 7/200, recon loss = 0.00932529
epoch : 8/200, recon loss = 0.00900926
epoch : 9/200, recon loss = 0.00853238
epoch : 10/200, recon loss = 0.00807931
epoch : 11/200, recon loss = 0.00779773
epoch : 12/200, recon loss = 0.00765350
epoch : 13/200, recon loss = 0.00758035
epoch : 14/200, recon loss = 0.00753677
epoch : 15/200, recon loss = 0.00750988
epoch : 16/200, recon loss = 0.00749236
epoch : 17/200, recon loss = 0.00746782
epoch : 18/200, recon loss = 0.00745426
epoch : 19/200, recon loss = 0.00743363
epoch : 20/200, recon loss = 0.00741140
epoch : 21/200, recon loss = 0.00739428
epoch : 22/200, recon loss = 0.00736950
epoch : 23/200, recon loss = 0.00733880
epoch : 24/200, recon loss = 0.00731051
epoch : 25/200, recon loss = 0.00726884
epoch : 2

In [ ]:
model.eval()
rmse_sum = 0

filled_data = model(missed_data.to(device))
filled_data = filled_data.cpu().detach().numpy()

rmse_sum = rmse_error(test_data, filled_data, cols, mask)

print(rmse_sum)

4.268289142753645


## Mean and Median Imputation

### By category

In [ ]:
imputed = X_test_mean
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

5.519246675443418


In [ ]:
imputed = X_test_median
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

5.730640468576924


### Across all foods

In [ ]:
imputed = X_test_mean_all
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

6.3815988582216585


In [ ]:
imputed = X_test_median_all
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

6.5167976604056905


# 198 Columns

In [ ]:
data_path = 'datasets/average datasets/byColumnsWithEmbeddings/198_columns_embeddings.csv'
num_embeddings = 50
num_columns = 198

In [ ]:
dataset = pd.read_csv(data_path, index_col=0)
dataset[dataset.notna().all(axis=1)].describe()

,fdc_id,1002,1003,1004,1005,1007,1008,1009,1010,1011,1012,1013,1014,1024,1032,1039,1050,1051,1062,1063,1075,1079,1082,1084,1085,1087,1089,1090,1091,1092,1093,1094,1095,1097,1098,1100,1101,1102,1103,1105,...,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
count,1.310500e+04,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,...,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000,13105.000000
mean,4.807517e+05,0.243665,1.942243,1.824697,0.164999,0.472742,1.999714,2.561062,0.147034,0.118187,0.120046,0.046356,0.008754,0.000476,0.681114,0.728577,0.049253,8.349769,8.369340,0.055411,0.002459,0.350355,0.005555,0.021892,0.106772,27.659061,0.476038,14.595597,55.746662,125.003892,133.394887,14.730179,0.384163,2.983579,0.077504,12.187135,0.149979,42.944411,0.840258,2.086684,...,-0.005261,-0.053342,0.032862,0.017149,0.053914,-0.050015,-0.006802,-0.001312,0.045397,-0.002815,-0.004867,0.015156,-0.002852,-0.007990,0.007587,-0.005105,0.013259,0.026135,-0.009025,-0.004237,-0.008990,-0.000290,0.009895,-0.005317,0.001269,0.006334,0.018435,0.004424,0.009392,-0.003170,-0.001646,-0.006830,0.003214,-0.000149,0.004693,-0.004719,0.000873,-0.001455,0.006576,0.003596
std,2.544018e+05,0.999601,6.583455,7.838466,2.905073,4.416751,25.783702,9.446549,2.828219,1.055735,1.016245,0.441966,0.129818,0.022249,14.844129,15.911796,1.303063,23.549114,107.911712,1.491246,0.034625,1.186155,0.082114,0.319893,2.306569,99.924214,1.483360,45.754175,153.432660,382.322293,1736.846193,56.459805,1.055340,11.848737,0.264892,237.511065,0.515558,187.200801,5.232485,29.375360,...,0.400175,0.345475,0.349559,0.332054,0.321282,0.292924,0.270047,0.284935,0.261685,0.269203,0.240661,0.243893,0.245143,0.248005,0.235594,0.205819,0.225068,0.212354,0.209393,0.205605,0.208641,0.187673,0.193694,0.181126,0.188895,0.180591,0.172620,0.162121,0.180927,0.171087,0.170579,0.162898,0.166174,0.156805,0.165867,0.149952,0.151873,0.145215,0.151300,0.148431
min,3.198770e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.933855,-0.969972,-1.097137,-0.920027,-1.006138,-1.000187,-0.798815,-0.687112,-0.668615,-0.801637,-0.726105,-0.698135,-0.734302,-0.848955,-0.669846,-0.604000,-0.624276,-0.615452,-0.597283,-0.806154,-0.673437,-0.584369,-0.800978,-0.701795,-0.663740,-0.697520,-0.542536,-0.560396,-0.753790,-0.567381,-0.571239,-0.533034,-0.571749,-0.680382,-0.648647,-0.538644,-0.550678,-0.580334,-0.568859,-0.518081
25%,3.256480e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [ ]:
data = dataset
train_data, test_data = train_test_split(data, test_size=test_size)

In [ ]:
train_data.describe()

,fdc_id,1002,1003,1004,1005,1007,1008,1009,1010,1011,1012,1013,1014,1024,1032,1039,1050,1051,1062,1063,1075,1079,1082,1084,1085,1087,1089,1090,1091,1092,1093,1094,1095,1097,1098,1100,1101,1102,1103,1105,...,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
count,1.048400e+04,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,...,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000,10484.000000
mean,4.794562e+05,0.241070,1.928870,1.821707,0.161388,0.461399,2.026159,2.518903,0.137158,0.118248,0.119258,0.046907,0.009217,0.000595,0.679321,0.732259,0.045408,8.340591,8.480275,0.046689,0.002474,0.344878,0.005790,0.022530,0.109062,27.787963,0.470211,14.459805,55.374952,123.405857,120.136112,14.461274,0.381541,2.938544,0.076159,12.661293,0.148089,42.606524,0.864411,2.053987,...,-0.004485,-0.053181,0.035190,0.016537,0.055051,-0.049332,-0.005589,-0.001322,0.047027,-0.003175,-0.003616,0.015436,-0.002492,-0.006390,0.007534,-0.006210,0.014451,0.026644,-0.009088,-0.003639,-0.008473,0.000794,0.010315,-0.003527,0.001452,0.006197,0.019294,0.003951,0.009452,-0.003930,-0.001654,-0.007448,0.003339,-0.001735,0.004656,-0.004393,0.000464,-0.000058,0.005960,0.003356
std,2.532723e+05,0.993474,6.563354,7.902323,2.846050,4.304038,25.760127,9.375699,2.657333,1.057639,1.015281,0.442991,0.135252,0.024874,14.940270,16.128424,1.207659,23.558816,107.815927,1.235632,0.035390,1.176192,0.082493,0.321813,2.326975,101.368809,1.478138,45.646184,153.060188,379.104934,1575.954298,56.017479,1.053653,11.787381,0.262443,244.012777,0.516502,187.724232,5.320714,29.167232,...,0.399808,0.345308,0.347456,0.332473,0.321406,0.292574,0.270887,0.285244,0.262761,0.269514,0.241773,0.244525,0.246485,0.247560,0.236307,0.205018,0.225697,0.213409,0.209506,0.205961,0.209026,0.187867,0.194700,0.181905,0.189907,0.180628,0.172939,0.162417,0.180656,0.171054,0.170816,0.163941,0.166224,0.157380,0.166602,0.150019,0.152127,0.145858,0.151741,0.148118
min,3.198770e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.933843,-0.969439,-1.097137,-0.920027,-1.006138,-1.000187,-0.798815,-0.687112,-0.668615,-0.801637,-0.726105,-0.698135,-0.734302,-0.831774,-0.661366,-0.604000,-0.624276,-0.615452,-0.597283,-0.806154,-0.673437,-0.584369,-0.787943,-0.701795,-0.663740,-0.697520,-0.529472,-0.536528,-0.753790,-0.557471,-0.571239,-0.533034,-0.571749,-0.680382,-0.648647,-0.538644,-0.550678,-0.580334,-0.559276,-0.518081
25%,3.256388e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [ ]:
test_data.describe()

,fdc_id,1002,1003,1004,1005,1007,1008,1009,1010,1011,1012,1013,1014,1024,1032,1039,1050,1051,1062,1063,1075,1079,1082,1084,1085,1087,1089,1090,1091,1092,1093,1094,1095,1097,1098,1100,1101,1102,1103,1105,...,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
count,2.621000e+03,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.0,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.00000,2621.000000,2621.000000,2621.000000,2621.000000,...,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000
mean,4.859338e+05,0.254044,1.995738,1.836654,0.179443,0.518111,1.893934,2.729699,0.186539,0.117944,0.123197,0.044151,0.006902,0.0,0.688287,0.713850,0.064632,8.386478,7.925601,0.090298,0.002396,0.372262,0.004617,0.019344,0.097612,27.143457,0.499348,15.138764,57.233499,131.396032,186.429989,15.805799,0.394651,3.163716,0.082882,10.29050,0.157542,44.295956,0.743647,2.217474,...,-0.008367,-0.053989,0.023549,0.019596,0.049367,-0.052746,-0.011657,-0.001276,0.038876,-0.001373,-0.009874,0.014033,-0.004292,-0.014387,0.007798,-0.000686,0.008489,0.024095,-0.008769,-0.006628,-0.011060,-0.004628,0.008216,-0.012476,0.000541,0.006882,0.014999,0.006314,0.009149,-0.000132,-0.001614,-0.004360,0.002712,0.006195,0.004842,-0.006025,0.002510,-0.007046,0.009037,0.004557
std,2.588545e+05,1.023870,6.664248,7.579117,3.130588,4.842051,25.882445,9.724790,3.427916,1.048284,1.020284,0.437916,0.105316,0.0,14.455965,15.017005,1.629949,23.514720,108.313495,2.238760,0.031385,1.225180,0.080585,0.312139,2.223482,93.940655,1.504124,46.188392,154.934092,394.939205,2268.675857,58.194010,1.062197,12.091657,0.274454,209.52813,0.511793,185.121625,4.863353,30.198858,...,0.401700,0.346211,0.357762,0.330427,0.320807,0.294364,0.266655,0.283749,0.257284,0.268003,0.236137,0.241392,0.239741,0.249726,0.232766,0.208973,0.222512,0.208108,0.208984,0.204199,0.207120,0.186865,0.189641,0.177829,0.184825,0.180475,0.171326,0.160948,0.182043,0.171217,0.169660,0.158665,0.166007,0.154350,0.162924,0.149703,0.150870,0.142503,0.149526,0.149701
min,3.199080e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,-0.933855,-0.969972,-0.973889,-0.889484,-0.844214,-0.975344,-0.795263,-0.683452,-0.613335,-0.801561,-0.678256,-0.625027,-0.734182,-0.848955,-0.669846,-0.603996,-0.549889,-0.615429,-0.597251,-0.639653,-0.673297,-0.525369,-0.800978,-0.603322,-0.656870,-0.595664,-0.542536,-0.560396,-0.528300,-0.567381,-0.546243,-0.532969,-0.571732,-0.606464,-0.621206,-0.538610,-0.515188,-0.532556,-0.568859,-0.486389
25%,3.256660e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,-0.300802,-0.333182,-0.211713,-0.219276,-0.162027,-0.220277,-0.199176,-

In [ ]:
indices = test_data['fdc_id']
indices

7626     333186
7678     333305
6030     330174
10633    748750
2944     325099
          ...  
3560     326060
7279     332543
4019     326847
1607     322877
2520     324378
Name: fdc_id, Length: 2621, dtype: int64

In [ ]:
del train_data['fdc_id']
del test_data['fdc_id']
columns = train_data.columns

In [ ]:
train_data = train_data.to_numpy()
test_data.fillna(0, inplace=True)
test_data = test_data.to_numpy()
data = dataset.values
rows, cols = data.shape
cols -= 1

y_test = test_data.copy()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
missed_data, missed_data_train, mask = missing_method(test_data, train_data, num_embeddings)
pd.DataFrame(missed_data).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247
count,2094.000000,2057.000000,2110.00000,2112.000000,2128.000000,2100.000000,2096.000000,2105.000000,2111.000000,2090.000000,2084.000000,2131.000000,2083.0,2117.000000,2081.000000,2127.000000,2111.000000,2074.000000,2093.000000,2095.000000,2075.000000,2098.000000,2067.000000,2142.000000,2113.000000,2087.000000,2087.000000,2151.000000,2086.000000,2085.000000,2079.000000,2103.000000,2122.000000,2118.000000,2119.000000,2125.000000,2092.000000,2124.000000,2122.000000,2103.000000,...,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000
mean,0.272364,1.939626,1.76163,0.173873,0.527603,1.359048,2.784098,0.168546,0.123302,0.108555,0.045763,0.007405,0.0,0.601323,0.590581,0.071180,8.551644,5.432015,0.074233,0.002869,0.364964,0.005434,0.017078,0.098735,27.946522,0.507834,14.693531,57.714551,133.066155,199.305995,16.016835,0.389544,2.962253,0.082634,9.752006,0.162012,45.254828,0.727260,2.500471,0.415597,...,-0.008367,-0.053989,0.023549,0.019596,0.049367,-0.052746,-0.011657,-0.001276,0.038876,-0.001373,-0.009874,0.014033,-0.004292,-0.014387,0.007798,-0.000686,0.008489,0.024095,-0.008769,-0.006628,-0.011060,-0.004628,0.008216,-0.012476,0.000541,0.006882,0.014999,0.006314,0.009149,-0.000132,-0.001614,-0.004360,0.002712,0.006195,0.004842,-0.006025,0.002510,-0.007046,0.009037,0.004557
std,1.096187,6.569316,7.18062,2.980519,4.902768,20.176795,9.807325,3.137919,1.126675,0.908371,0.437763,0.113087,0.0,13.862775,13.577470,1.766844,23.868884,80.292984,2.237338,0.034784,1.214510,0.088752,0.289027,2.368098,98.178407,1.521939,45.325499,154.395943,398.289975,2395.089148,58.614368,1.049673,11.754408,0.273274,190.699412,0.522117,187.593139,4.862275,33.054742,10.612560,...,0.401700,0.346211,0.357762,0.330427,0.320807,0.294364,0.266655,0.283749,0.257284,0.268003,0.236137,0.241392,0.239741,0.249726,0.232766,0.208973,0.222512,0.208108,0.208984,0.204199,0.207120,0.186865,0.189641,0.177829,0.184825,0.180475,0.171326,0.160948,0.182043,0.171217,0.169660,0.158665,0.166007,0.154350,0.162924,0.149703,0.150870,0.142503,0.149526,0.149701
min,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.933855,-0.969972,-0.973889,-0.889484,-0.844214,-0.975344,-0.795263,-0.683452,-0.613335,-0.801561,-0.678256,-0.625027,-0.734182,-0.848955,-0.669846,-0.603996,-0.549889,-0.615429,-0.597251,-0.639653,-0.673297,-0.525369,-0.800978,-0.603322,-0.656870,-0.595664,-0.542536,-0.560396,-0.528300,-0.567381,-0.546243,-0.532969,-0.571732,-0.606464,-0.621206,-0.538610,-0.515188,-0.532556,-0.568859,-0.486389
25%,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.300802,-0.333182,-0.211713,-0.219276,-0.162027,-0.220277,-0.199176,-0.173755,-0.138855,-0.157256,-0.125563,-0.140941,-0.126020,-0.188958,-

In [ ]:
# traditional methods test sets - impute across all foods
X_train = pd.DataFrame(missed_data_train, columns = columns)
X_test_mean_all = pd.DataFrame(missed_data, columns = columns)
X_test_median_all = pd.DataFrame(missed_data, columns = columns)

X_test_mean_all = impute_traditional_all_foods(X_train, X_test_mean_all, 'mean')
X_test_median_all = impute_traditional_all_foods(X_train, X_test_median_all, 'median')

In [ ]:
# Traditional methods test sets - impute by food category
X_test_mean = pd.DataFrame(missed_data, columns = columns, index=indices)
X_test_median = X_test_mean.copy()
df_mean_cat = df_mean[df_mean['fdc_id'].isin(indices)]
X_test_mean = impute_traditional_by_category(X_test_mean, df_mean_cat, indices)
df_median_cat = df_median[df_median['fdc_id'].isin(indices)]
X_test_median = impute_traditional_by_category(X_test_median, df_median_cat, indices)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)
X_test_mean = scaler.transform(X_test_mean)
X_test_median = scaler.transform(X_test_median)
X_test_mean_all = scaler.transform(X_test_mean_all)
X_test_median_all = scaler.transform(X_test_median_all)

y_test = scaler.transform(y_test)

In [ ]:
# datasets without embeddings for autoencoder
train_data_noEmbeddings = pd.DataFrame(train_data, columns = columns)
train_data_noEmbeddings = np.array(train_data_noEmbeddings.iloc[:, :-num_embeddings]) # df = df.iloc[: , :-1]
test_mean_noEmbeddings = np.array(pd.DataFrame(X_test_mean, columns = columns).iloc[:, :-num_embeddings])
test_median_noEmbeddings = np.array(pd.DataFrame(X_test_median, columns = columns).iloc[:, :-num_embeddings])
test_mean_noEmbeddings_all = np.array(pd.DataFrame(X_test_mean_all, columns = columns).iloc[:, :-num_embeddings])
test_median_noEmbeddings_all = np.array(pd.DataFrame(X_test_median_all, columns = columns).iloc[:, :-num_embeddings])

In [ ]:
missed_data = X_test_mean_all
missed_data = torch.from_numpy(missed_data).float()

train_data = torch.from_numpy(train_data).float()

train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=batch_size,
                                           shuffle=True)

In [ ]:
missed_data_noEm = test_mean_noEmbeddings_all
missed_data_noEm = torch.from_numpy(missed_data_noEm).float()

train_data_noEmbeddings = torch.from_numpy(train_data_noEmbeddings).float()

train_loader_noEm = torch.utils.data.DataLoader(dataset=train_data_noEmbeddings,
                                           batch_size=batch_size,
                                           shuffle=True)

## Autoencoder - without embeddings

In [ ]:
model = Autoencoder(dim=cols-num_embeddings).to(device)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
training(num_epochs, model, train_loader_noEm, criterion, optimizer)

epoch : 1/200, recon loss = 0.00376239
epoch : 2/200, recon loss = 0.00293869
epoch : 3/200, recon loss = 0.00290759
epoch : 4/200, recon loss = 0.00288642
epoch : 5/200, recon loss = 0.00285750
epoch : 6/200, recon loss = 0.00281565
epoch : 7/200, recon loss = 0.00275599
epoch : 8/200, recon loss = 0.00266495
epoch : 9/200, recon loss = 0.00252990
epoch : 10/200, recon loss = 0.00233611
epoch : 11/200, recon loss = 0.00214065
epoch : 12/200, recon loss = 0.00196707
epoch : 13/200, recon loss = 0.00185716
epoch : 14/200, recon loss = 0.00178805
epoch : 15/200, recon loss = 0.00174686
epoch : 16/200, recon loss = 0.00172417
epoch : 17/200, recon loss = 0.00171114
epoch : 18/200, recon loss = 0.00169873
epoch : 19/200, recon loss = 0.00168953
epoch : 20/200, recon loss = 0.00168483
epoch : 21/200, recon loss = 0.00168238
epoch : 22/200, recon loss = 0.00167891
epoch : 23/200, recon loss = 0.00167378
epoch : 24/200, recon loss = 0.00167387
epoch : 25/200, recon loss = 0.00166987
epoch : 2

In [ ]:
model.eval()
rmse_sum = 0

filled_data = model(missed_data_noEm.to(device))
filled_data = filled_data.cpu().detach().numpy()

rmse_sum = rmse_error(test_data, filled_data, cols-num_embeddings, mask)

print(rmse_sum)

5.215949090468736


## Autoencoder - with embeddings

In [ ]:
model = Autoencoder(dim=cols).to(device)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
training(num_epochs, model, train_loader, criterion, optimizer)

epoch : 1/200, recon loss = 0.01425203
epoch : 2/200, recon loss = 0.00729474
epoch : 3/200, recon loss = 0.00727854
epoch : 4/200, recon loss = 0.00726253
epoch : 5/200, recon loss = 0.00724003
epoch : 6/200, recon loss = 0.00721195
epoch : 7/200, recon loss = 0.00717784
epoch : 8/200, recon loss = 0.00713054
epoch : 9/200, recon loss = 0.00706220
epoch : 10/200, recon loss = 0.00696389
epoch : 11/200, recon loss = 0.00682904
epoch : 12/200, recon loss = 0.00666035
epoch : 13/200, recon loss = 0.00648561
epoch : 14/200, recon loss = 0.00634780
epoch : 15/200, recon loss = 0.00624832
epoch : 16/200, recon loss = 0.00618158
epoch : 17/200, recon loss = 0.00613688
epoch : 18/200, recon loss = 0.00610990
epoch : 19/200, recon loss = 0.00608614
epoch : 20/200, recon loss = 0.00607367
epoch : 21/200, recon loss = 0.00606013
epoch : 22/200, recon loss = 0.00605006
epoch : 23/200, recon loss = 0.00604083
epoch : 24/200, recon loss = 0.00603483
epoch : 25/200, recon loss = 0.00602478
epoch : 2

In [ ]:
model.eval()
rmse_sum = 0

filled_data = model(missed_data.to(device))
filled_data = filled_data.cpu().detach().numpy()

rmse_sum = rmse_error(test_data, filled_data, cols, mask)

print(rmse_sum)

5.712720665341518


## Mean and Median Imputation

### By category

In [ ]:
imputed = X_test_mean
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

6.562400490537268


In [ ]:
imputed = X_test_median
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

6.729803902928729


### Across all foods

In [ ]:
imputed = X_test_mean_all
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

7.4467183945414215


In [ ]:
imputed = X_test_median_all
error = rmse_error(y_test, imputed, cols-num_embeddings, mask)
print(error)

7.573344537892341
